This notebook might not be usable "out of the box" and is mainly meant to give an idea how the data was calculated from the NMR bundles and the MD ensembles.

# Import modules

In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
import barnaba as bb
from barnaba import definitions
import subprocess as sub
import regex as re
import mdtraj as md

# Define Functions

In [2]:
def load( inp ):
    pin = open( inp, "rb" )
    return pkl.load( pin )

In [3]:
def save( outfile, results ):
    with open( outfile + ".pkl", "wb" ) as fp:
        pkl.dump( results, fp )

In [4]:
### HELPER FUNCTIONS TO CALCULATE NOE DISTANCES ###

# helper functions, subsititute strings. This is because the name of hydrogens is a mess
# alt = {"H2'":"1H2'","H5''":"2H5'","H5'":"1H5'","HO2'":"2HO'","H5\"":"2H5'","H5'2":"2H5'","H5'1":"1H5'"}
alt = {"H2'":"H2'1","H5''":"H5'2","H5'":"H5'1","HO2'":"HO'2"}

def sub(ss):
    at = "H" + ss.split("H")[1]
    if(at in alt):
        at = alt[at]
    return ss.split("H")[0] + at

# read experimental datafile and returns a list of labels and experimental values
def read_exp(f_exp):
    labels = []
    vals = []
    with open(f_exp) as fh:
        for line in fh:
            if("#" not in line):
                r1 = line.split()[0].split("-")[0]
                print(r1)
                r2 = line.split()[0].split("-")[1]
                print(r2)
                v1 = np.sort([r1,r2])
                print(v1)
                qq = v1[0] +"/"+ v1[1]
                
                if(qq in labels):
                    print("# DUPLICATE. Skipping data.."),
                    print(qq,vals[labels.index(qq)], line),
                else:
                    vals.append([float(line.split()[1]),float(line.split()[2])])
                    labels.append(qq)
    return labels,vals

# get labels from df column (Assignment) 
def get_labels(df, col):
    labels = []
    for asm in df.iloc[:,col]:
        r1 = asm.split()[0].split("-")[0]
        r2 = asm.split()[0].split("-")[1]
        v1 = np.sort([r1,r2])
        qq = v1[0] +"/"+ v1[1]
        labels.append(qq)
    return labels

# find indeces in topology corresponding to labels in experimental datafile
def get_idxs(labels,top):

    atoms = []
    for atom in top.atoms:
        aa = str(atom).split("-")[1]
        if(aa in alt): aa = alt[aa]
        atoms.append("%s%s" % (str(atom).split("-")[0],aa))
    pairs = []
    for el in labels:
        ss  = el.split("/")
        at1 = sub(ss[0])
        at2 = sub(ss[1])
        if(at1 in atoms and at2 in atoms):
            pairs.append([atoms.index(at1),atoms.index(at2)])
        else:
            print("# Warning: Either %s or %s are missing" % (at1,at2))
            return 0
    if len(pairs) != len(labels):
        print("# Found only %d pairs out of %d" % (len(pairs),len(labels)))
    return np.array(pairs)

def group_by_heading( some_source ):
    buffer= []
    for line in some_source:
        if line.startswith( " ASSI" ):
            if buffer: yield buffer
            buffer= [ line.strip().strip(')') ]
        else:
            buffer.append( line.strip().strip(')') )
    yield buffer

# From NMR bundles

Back-calculate experimental observables from ```Bundle_E.pdb```, ```Bundle_F.pdb``` and ```1RNG.pdb```.

Since some of the forward models need certain PDB formats we have to reformat the PDB files.

In [1]:
# NMR bundle directory:
bundles_dir = 'nmrbundle_data/'

# Bundle identifiers:
bundles = ['E', 'F', '1RNG']

## RDCs

We use the pf1-phage prediction method implemented in *PALES** to back-calculate RDCs. In order to do that we use the script ```calc_rdc_bundles.py``` which submits each frame to *PALES*, predicts the alignment tensor and calculates RDCs, parses the output and saves the D values in a Pickle file.

The actual command line to run *PALES is*: ```pales-linux -inD exp_data/exp_rdc_pales.tab -pdb {pdb_tmp} -outD {outd_tmp} -pf1 -H -wv 0.05```


*Zweckstetter, M. NMR: Prediction of molecular alignment from structure using the PALES software. Nat. Protoc. 3, 679–690 (2008).

In [174]:
rdc_exp_input = pd.read_csv('exp_data/exp_rdc_pales.tab', names=['RESID_I', 'RESNAME_I', 'ATOMNAME_I', 'RESID_J', 'RESNAME_J',
       'ATOMNAME_J', 'D', 'DD', 'W'], delim_whitespace=True, skiprows=5)
# all residues:
rdc_exp_labels_all = [f"{row[1]['RESNAME_I'][0]}{row[1]['RESID_I']}_{row[1]['ATOMNAME_I']}-{row[1]['ATOMNAME_J']}" for row in rdc_exp_input.iterrows()]
# loop residues:
rdc_exp_labels_loop = [f"{row[1]['RESNAME_I'][0]}{row[1]['RESID_I']}_{row[1]['ATOMNAME_I']}-{row[1]['ATOMNAME_J']}" for row in rdc_exp_input.iterrows() if row[1]['RESID_I'] in [5,6,7,8,9,10]]

In [60]:
for bundle in bundles:
    process = sub.Popen(f'python {bundles_dir}/calc_rdc_traj.py {bundle}', shell=True, stdin=sub.PIPE, stdout=sub.PIPE, universal_newlines=True)
    process.wait() # needed to run one process at a time (because of using temporary files etc.)



 *--------------------------------------------------------------------* 
 |               PALES                      (C) M. Zweckstetter 2000  | 
 *--------------------------------------------------------------------*

 

/storage1/kummerer/TETRALOOPS/CUUG/exp_data/paper_bundles//rdcs/frame_0.pdb, 383 Atoms, Residues 2 to 13.
/storage1/kummerer/TETRALOOPS/CUUG/exp_data/FINAL_DATA_18112022/cuugrdc_all308.tab, 30 Couplings, Residues 1 to 13.
REMARK Orientation 2196 of 2196
REMARK Memory Allocation Status. Reused: 656 In Use: 2887796


 *--------------------------------------------------------------------* 
 |               PALES                      (C) M. Zweckstetter 2000  | 
 *--------------------------------------------------------------------*

 

/storage1/kummerer/TETRALOOPS/CUUG/exp_data/paper_bundles//rdcs/frame_1.pdb, 383 Atoms, Residues 2 to 13.
/storage1/kummerer/TETRALOOPS/CUUG/exp_data/FINAL_DATA_18112022/cuugrdc_all308.tab, 30 Couplings, Residues 1 to 13.
REMARK Orientat

Parse the output and write BME readable files:

In [74]:
for bundle in bundles:
    df = load(f'{bundles_dir}/nmrbundle_{bundle}_d_df_pf1.pkl')
    for i in rdc_exp_labels_all: # missing residues in 1RNG
        if i not in df.columns:
            df[i] = np.full((len(df)), np.nan)
    
    # All res:
    df[rdc_exp_labels_all].to_csv(f'{bundles_dir}/nmrbundle_{bundle}_rdc_all_bme.dat', header=False, sep='\t', na_rep=np.nan)
    # Loop only:
    df[rdc_exp_labels_loop].to_csv(f'{bundles_dir}/nmrbundle_{bundle}_rdc_loop_bme.dat', header=False, sep='\t', na_rep=np.nan)

The predicted RDCs need to be scaled to the experimental values before comparing them. To reduce the effect of over-fitting we scale the RDCs based on all measurements and since we treat NMR structures as a bundle of structures rather than an ensemble we do the scaling for all individual structures.

In [75]:
for bundle in bundles:
    rdc_bundle = np.loadtxt(f'{bundles_dir}/nmrbundle_{bundle}_rdc_all_bme.dat')
    L = np.zeros(len(rdc_bundle))
    for i in range(len(L)):
        masked = np.ma.array(rdc_bundle[i,1:], mask=np.isnan(rdc_bundle[i,1:])) # mask NaNs (only present in 1RNG due to missing/different residues in the stem)
        L[i] = np.sum(np.array(rdc_exp_input['D'])*masked)/np.sum(masked*masked)
    # print(L)
    np.save(f'{bundles_dir}/nmrbundle_{bundle}_rdc_all_bme_L', L)

[-0.42217735 -0.47304937 -0.42535987 -0.38887352 -0.42488561 -0.42047199
 -0.42816358 -0.42314775 -0.4282601  -0.44315126 -0.41305291 -0.42628107
 -0.49120921 -0.42238565 -0.49735445 -0.45195944 -0.38918149 -0.45317832
 -0.42410985 -0.42566778]
[-0.3559229  -0.37141832 -0.4071307  -0.42057445 -0.4817862  -0.4574938
 -0.36979183 -0.34962068 -0.36812801 -0.40707106 -0.37666958 -0.38420109
 -0.33849285 -0.40938766 -0.38990872 -0.3528908  -0.36597962 -0.36588416
 -0.38999744 -0.40099118]
[-0.49517812 -0.5017432  -0.47475832 -0.50159652 -0.51723057 -0.51994268
 -0.47711362 -0.48615831 -0.45209726 -0.54786246 -0.46050511 -0.50893858
 -0.45273783 -0.44680576 -0.50977555 -0.47868166 -0.45465672 -0.4379871
 -0.46705837 -0.4584089 ]
[-0.44475955 -0.43542508 -0.42223536 -0.49241141 -0.48400591 -0.45570632
 -0.47337935 -0.44771636 -0.40395842 -0.4375156  -0.4809261  -0.41498865
 -0.47143533 -0.48384091 -0.476374   -0.4863376  -0.45259264 -0.41579333
 -0.46093421 -0.41923211]
[-0.42334988 -0.472367

## $^3$J-couplings

We use Barnaba* to calculate the $^3$J scalar couplings 2H5H4, H1H2, H2H3, H3P, C4Pb, 1H5P, 1H5H4, C4Pe, 2H5P and H3H4.

Definition: $A cos^2 (\theta + \phi) + B cos (\theta + \phi) + C$

*Bottaro, S. et al. Barnaba: software for analysis of nucleic acid structures and trajectories. Rna 25, 219–231 (2019)

In [65]:
# A (Hz), B (Hz), C (Hz), _, phi (rad)
bb.definitions.couplings_karplus

{'H1H2': [9.67, -2.03, 0.0, 0.0, 0.0],
 'H2H3': [9.67, -2.03, 0.0, 0.0, 0.0],
 'H3H4': [9.67, -2.03, 0.0, 0.0, 0.0],
 '1H5P': [15.3, -6.1, 1.6, 0.0, -2.094395],
 '2H5P': [15.3, -6.1, 1.6, 0.0, 2.094395],
 'C4Pb': [6.9, -3.4, 0.7, 0.0, 0.0],
 '1H5H4': [9.7, -1.8, 0.0, 0.0, -2.094395],
 '2H5H4': [9.7, -1.8, 0.0, 0.0, 0.0],
 'H3P': [15.3, -6.1, 1.6, 0.0, 2.094395],
 'C4Pe': [6.9, -3.4, 0.7, 0.0, 0.0],
 'H1C2/4': [4.7, 2.3, 0.1, 0.0, -1.0471975],
 'H1C6/8': [4.5, -0.6, 0.1, 0.0, -1.0471975]}

In [99]:
j3_exp_labels = list(pd.read_csv('exp_data/exp_j3_bme.dat', delim_whitespace=True).index) # ONLY EXTENDED-LOOP RESIDUES!!!
j3_exp_couplings = list(set([ i.split('-')[-1] for i in j3_exp_labels ]))

barnaba_to_exp = {'H1H2':"H1',H2'", 'H2H3':"H2',H3'", 'H3H4':"H3',H4'",\
                '1H5P':"H5'i,Pi", '2H5P':"H5''i,Pi",\
                'C4Pb':"C4'i,Pi", '1H5H4':"NA", '2H5H4':"NA", \
                'H3P':"H3'i,Pi+1" ,'C4Pe':"C4'i,Pi+1", \
                'H1C2/4':"NA", 'H1C6/8':"NA"}

# the shape of couplings is (nframes, nresidues, ncouplings)

for bundle in bundles:
    traj_file   = f'{bundles_dir}/Bundle_{bundle}_gmx.pdb'
    top_file    = f'{bundles_dir}/Bundle_{bundle}_gmx.pdb'

    couplings,residues = bb.jcouplings(traj_file, topology=top_file, couplings=j3_exp_couplings)
    print( f'Calculated jcouplings for {j3_exp_couplings}.' )
    print( couplings.shape )

    couplings_avg = np.average( couplings, 0 )
    print(couplings_avg.shape)
    
    j3_calc_ = pd.DataFrame()
    for i,ii in enumerate([ i.split('_')[0]+i.split('_')[1] for i in rr ]):
        for j,jj in enumerate(j3_exp_couplings):
            j3_calc_[f'{ii}-{jj}'] = couplings[:,i,j]

    ll = []
    for i in j3_exp_labels:
        ll.append(j3_calc_[i])
    pd.DataFrame(ll).T.to_csv(f'{bundles_dir}/nmrbundle_{bundle}_j3_loop_bme.dat', sep=' ', header=False)

# Loading /storage1/kummerer/TETRALOOPS/CUUG/exp_data/paper_bundles//Bundle_A_gmx.pdb 
/lindorffgrp-isilon/kummerer/software/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


Calculated jcouplings for ['1H5H4', 'C4Pe', '1H5P', 'H3P', 'C4Pb', '2H5P', 'H3H4', '2H5H4', 'H2H3', 'H1H2'].
(20, 14, 10)
(14, 10)


# Loading /storage1/kummerer/TETRALOOPS/CUUG/exp_data/paper_bundles//Bundle_B_gmx.pdb 


Calculated jcouplings for ['1H5H4', 'C4Pe', '1H5P', 'H3P', 'C4Pb', '2H5P', 'H3H4', '2H5H4', 'H2H3', 'H1H2'].
(20, 14, 10)
(14, 10)


# Loading /storage1/kummerer/TETRALOOPS/CUUG/exp_data/paper_bundles//Bundle_C_gmx.pdb 


...

Calculated jcouplings for ['1H5H4', 'C4Pe', '1H5P', 'H3P', 'C4Pb', '2H5P', 'H3H4', '2H5H4', 'H2H3', 'H1H2'].
(20, 14, 10)
(14, 10)


# Loading /storage1/kummerer/TETRALOOPS/CUUG/exp_data/paper_bundles//Bundle_E_gmx.pdb 


Calculated jcouplings for ['1H5H4', 'C4Pe', '1H5P', 'H3P', 'C4Pb', '2H5P', 'H3H4', '2H5H4', 'H2H3', 'H1H2'].
(20, 14, 10)
(14, 10)


# Loading /storage1/kummerer/TETRALOOPS/CUUG/exp_data/paper_bundles//Bundle_F_gmx.pdb 
# Loading /storage1/kummerer/TETRALOOPS/CUUG/exp_data/paper_bundles//Bundle_1RNG_gmx.pdb 


Calculated jcouplings for ['1H5H4', 'C4Pe', '1H5P', 'H3P', 'C4Pb', '2H5P', 'H3H4', '2H5H4', 'H2H3', 'H1H2'].
(20, 14, 10)
(14, 10)
Calculated jcouplings for ['1H5H4', 'C4Pe', '1H5P', 'H3P', 'C4Pb', '2H5P', 'H3H4', '2H5H4', 'H2H3', 'H1H2'].
(5, 12, 10)
(12, 10)


## CCRs

We use the script ```calc_ccr.py``` to back-calculate CCRs from pdb files or trajectories
Important notes:
- The PDB naming has to be in a specific format (check the script if in doubt).
- Here, $\Gamma$-HCP (C4p-P, C3p-P-plus, C4p-P-plus) and $\Gamma$-HCNCH (C1-CC) are measured at 600 MHz (14.09 T) while $\Gamma$-HCCH (C1-C2, C3-C4) is measured at 700 MHz (16.44 T) which means that in theory the script has to be executed twice and B0 has to be adjusted accordingly. However, $\Gamma$-HCCH couplings are not influenced by B0 which is why we can ignore this for now.

Load experimental data to get labels:

In [54]:
ccr_exp_labels_loop  = list(pd.read_csv('exp_data/exp_ccr.dat', delim_whitespace=True, usecols=[0]).index)
ccr_exp_loop  = np.loadtxt('exp_data/exp_ccr.dat', usecols=[1,2])

33

Calculate CCRs from bundles:

In [19]:
# Check the script and change output name etc.
for bundle in bundles:
    process = sub.Popen(f'python /storage1/kummerer/scripts/calc_ccr.py Bundle_{bundle}_gmx_ccr.pdb', shell=True, stdin=sub.PIPE, stdout=sub.PIPE, universal_newlines=True)
    process.wait() # needed to run one process at a time (because of using temporary files etc.)

Parse calculated data and write to BME file:

In [92]:
# for bundle in bundles:
#     tmp_ccr_600 = pd.read_csv(f'{bundles_dir}/ccr/600MHz/Bundle_{bundle}_gmx_ccr_calc.dat', delim_whitespace=True, index_col=0)
#     tmp_ccr_loop_600 = tmp_ccr_600[ccr_exp_labels_loop]
    # tmp_ccr_700 = pd.read_csv(f'{bundles_dir}/ccr/700MHz/Bundle_{bundle}_gmx_ccr_calc.dat', delim_whitespace=True, index_col=0)
    # Check if all data points are contained in the bundle:
    # print(list(tmp_ccr_loop.columns) == ccr_lbl)
    # # Combine 700 MHz data for gamma-HCCH (C1-C2, C3-C4) couplings with the rest at 600 MHz:
    # for lbl in [ i for i in ccr_exp_labels_loop if i.split(':')[1] in ['C1-C2', 'C3-C4'] ]:
    #     tmp_ccr_600[lbl] = tmp_ccr_700[lbl]
    # Save as BME readable file:
    # tmp_ccr_loop_600.to_csv(f'{bundles_dir}/ccr/Bundle_{bundle}_gmx_ccr_calc_loop_bme.dat', header=False, sep=' ')

True
True
True
True
True
True
True


NOTE: Don't forget about adding nans for missing measurements in 1RNG when compiling a data file for all residues (including stem).

## NOEs

We load NOEs from an ARIA output file after they have been converted to distances (\AA). For this, we use the NOEs from bundle F because those have been restraint the most by other types of data. Generally, they don't differ significantly between the different bundles.

Load and parse experimental NOE distance file:

In [10]:
seq = {1:'G', 2:'G', 3:'C', 4:'A', 5:'G', 6:'C', 7:'U', 8:'U', 9:'G', 10:'C', 11:'U', 12:'G', 13:'C', 14:'C'}

df_noe_exp = pd.DataFrame()

with open('exp_data/exp_noe.tbl') as f: # use NOEs from bundle F
    
    for heading_and_lines in group_by_heading( f ):
        heading= heading_and_lines[0]
        lines= heading_and_lines[1:]
        
        resid1 = int(re.search('resid (\d+)', lines[0] ).group(1))
        name1 = re.search('name (.+)', lines[0] ).group(1).strip()
        resid2 = int(re.search('resid (\d+)', lines[1] ).group(1))
        name2 = re.search('name (.+)', lines[1] ).group(1).strip()
        
        row = {'Assignment': f'{seq[resid1]}{resid1}{name1}-{seq[resid2]}{resid2}{name2}', 'distance':float(lines[2].split()[0]), 'upper_err':float(lines[2].split()[1]), 'lower_err':float(lines[2].split()[2])}
        df_noe_exp = df_noe_exp.append(row, ignore_index=True, verify_integrity=True)

Make separate dfs for loop NOEs:

In [19]:
residues = ['G1', 'G2', 'C3', 'A4', 'G5', 'C6', 'U7', 'U8', 'G9', 'C10', 'U11', 'G12', 'C13', 'C14']
loop = ['G5','C6', 'U7', 'U8', 'G9','C10']

a_loop, a_stem, a_both = [], [], []
for i,n in df_noe_exp[:].iterrows():
    a = n['Assignment'].split('-')
    r1 = re.split('(^[A-Z]\d+)', a[0])[1]
    r2 = re.split('(^[A-Z]\d+)', a[1])[1]
    if r1 in loop and r2 in loop:
        a_loop.append(n['Assignment'])
    # elif r1 in loop or r2 in loop:
    #     a_both.append(n['Assignment'])
    # else:
    #     a_stem.append(n['Assignment'])
print(f'Loop NOEs: {len(a_loop)}')
# print( f'Loop NOEs: {len(a_loop)}\nStem NOEs: {len(a_stem)}\nIntersect: {len(a_both)}' )

df_noe_exp_loop = df_noe_exp.loc[df_noe_exp.Assignment.isin(a_loop)]

Loop NOEs: 76


Calc distances from the bundles (1RNG has to be done separately due to missing/different residues):

In [21]:
# Calc distances from the bundles (1RNG has to be one separately due to missing/different residues)
for bundle in bundles[:-1]: 
    print(bundle)
    traj_ = md.load_pdb(f'nmrbundle_data/Bundle_{bundle}_gmx.pdb')
    # labels = get_labels(df_noe_exp, 0) # For all NOEs
    labels = get_labels(df_noe_exp_loop, 0)
    pairs = get_idxs( labels, traj_.topology )
    # calculate distances multiply by 10 to convert to angs
    dists = 10.0*md.compute_distances(traj_,pairs)
    df_noe = pd.DataFrame(dists)

    df_noe.columns = list(df_noe_exp_loop.Assignment)
    # Save to BME format:
    df_noe[a_loop].to_csv(f'nmrbundle_data/Bundle_{bundle}_dists_loop.dat', sep='\t', header=False)

A
B
C
D
E
F


Calc distances from 1RNG -> only do for loop residues:

In [22]:
# Calc distances from 1RNG -> only do for loop residues:
# tmp_labels = pd.DataFrame()
# tmp_labels[0] = a_loop

for bundle in bundles[-1:]: #['A', 'B', 'C']:
    traj_ = md.load_pdb(f'nmrbundle_data/Bundle_{bundle}_gmx_rn.pdb')
    print(bundle)
    labels = get_labels(df_noe_exp_loop, 0)
    pairs = get_idxs( labels, traj_.topology )
    # calculate distances multiply by 10 to convert to angs
    dists = 10.0*md.compute_distances(traj_,pairs)
    df_noe = pd.DataFrame(dists)

    df_noe.columns = a_loop
    # Save to BME format:
    df_noe[a_loop].to_csv(f'nmrbundle_data/Bundle_{bundle}_dists_loop.dat', sep='\t', header=False)

1RNG


Write BME readable file for the experimental loop noes:

In [24]:
df_noe_exp_loop_tobme = df_noe_exp_loop.copy().drop(columns=['lower_err', 'upper_err'])
df_noe_exp_loop_tobme['avg_err'] = ((df_noe_exp_loop['lower_err']+df_noe_exp_loop['upper_err'])/2)

In [28]:
with open(f'exp_data/exp_noe_loop_bme.dat', 'w') as f:
    f.write('# DATA=NOE POWER=6\n')
    df_noe_exp_loop_tobme.to_csv(f, index= False, header = False, sep = '\t', float_format='%.2f') #df_noe_exp_tobme[df_noe_exp_tobme['Assignment'].isin(a_loop)]

# From MD ensembles

## RDCs

We use the pf1-phage prediction method implemented in *PALES** to back-calculate RDCs. In order to do that we use the script ```calc_rdc_traj.py``` which submits each frame to *PALES*, predicts the alignment tensor and calculates RDCs, parses the output and saves the D values in a Pickle file.

The actual command line to run *PALES is*: ```pales-linux -inD exp_data/exp_rdc_pales.tab -pdb {pdb_tmp} -outD {outd_tmp} -pf1 -H -wv 0.05```


*Zweckstetter, M. NMR: Prediction of molecular alignment from structure using the PALES software. Nat. Protoc. 3, 679–690 (2008).

In [13]:
# After running the script above:
ll = []
for i in range(0,20):
    df = load(f'cuug_{i}/d_df_pf1.pkl') # Change
    ll.append(df)
ddff = pd.concat(ll)#, ignore_index=True)
d_md = ddff.reset_index(drop=True)[list(d_exp_rdc.index)]
d_md_loop = d_md[list(d_exp_rdc_loop.index)]
# All to BME
# d_md.to_csv(f'calc_data/calc_rdc_bme.dat', header=False, sep='\t')
# Loop to BME:
# d_md_loop.to_csv(f'calc_data/calc_rdc_loop_bme.dat', header=False, sep='\t')

## $^3$J-couplings

We use Barnaba* to calculate the $^3$J scalar couplings 2H5H4, H1H2, H2H3, H3P, C4Pb, 1H5P, 1H5H4, C4Pe, 2H5P and H3H4.

Definition: $A cos^2 (\theta + \phi) + B cos (\theta + \phi) + C$

*Bottaro, S. et al. Barnaba: software for analysis of nucleic acid structures and trajectories. Rna 25, 219–231 (2019)

In [10]:
j3_exp_labels = list(pd.read_csv('exp_data/exp_j3_bme.dat', delim_whitespace=True).index) # ONLY EXTENDED-LOOP RESIDUES!!!
j3_exp_couplings = list(set([ i.split('-')[-1] for i in j3_exp_labels ]))

barnaba_to_exp = {'H1H2':"H1',H2'", 'H2H3':"H2',H3'", 'H3H4':"H3',H4'",\
                '1H5P':"H5'i,Pi", '2H5P':"H5''i,Pi",\
                'C4Pb':"C4'i,Pi", '1H5H4':"NA", '2H5H4':"NA", \
                'H3P':"H3'i,Pi+1" ,'C4Pe':"C4'i,Pi+1", \
                'H1C2/4':"NA", 'H1C6/8':"NA"}

# the shape of couplings is (nframes, nresidues, ncouplings)

traj_file   = f'concat_traj_nopbc.xtc'
top_file    = f'initial_nopbc_mdtraj.pdb'

couplings,residues = bb.jcouplings(traj_file, topology=top_file, couplings=j3_exp_couplings)
print( f'Calculated jcouplings for {j3_exp_couplings}.' )
print( couplings.shape )

couplings_avg = np.average( couplings, 0 )
print(couplings_avg.shape)

j3_calc_ = pd.DataFrame()
for i,ii in enumerate([ i.split('_')[0]+i.split('_')[1] for i in rr ]):
    for j,jj in enumerate(j3_exp_couplings):
        j3_calc_[f'{ii}-{jj}'] = couplings[:,i,j]

ll = []
for i in j3_exp_labels:
    ll.append(j3_calc_[i])
pd.DataFrame(ll).T.to_csv(f'calc_data/calc_j3_bme.dat', sep=' ', header=False)

# Loading concat_traj_nopbc.xtc 


Calculated jcouplings for ['H1H2', '1H5P', '1H5H4', 'C4Pb', '2H5P', 'H2H3', 'H3P', 'H3H4', 'C4Pe', '2H5H4'].
(20100, 14, 10)
(14, 10)


/lindorffgrp-isilon/kummerer/software/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


## CCRs

We use the script ```calc_ccr.py``` to back-calculate CCRs from pdb files or trajectories
Important notes:
- The PDB naming has to be in a specific format (check the script if in doubt).
- Here, $\Gamma$-HCP (C4p-P, C3p-P-plus, C4p-P-plus) and $\Gamma$-HCNCH (C1-CC) are measured at 600 MHz (14.09 T) while $\Gamma$-HCCH (C1-C2, C3-C4) is measured at 700 MHz (16.44 T) which means that in theory the script has to be executed twice and B0 has to be adjusted accordingly. However, $\Gamma$-HCCH couplings are not influenced by B0 which is why we can ignore this for now.

## NOEs

In [30]:
dfs_noe = {}
labels = get_labels(df_noe_exp_loop, 0)

traj_file   = f'concat_traj_nopbc.xtc'
top_file    = f'initial_nopbc_mdtraj.pdb'

traj  = md.load( traj_file, top = top_file ) # load traj
pairs = get_idxs( labels, traj.topology )

# calculate distances multiply by 10 to convert to angs
dists = 10.0*md.compute_distances(traj,pairs)
df_noe = pd.DataFrame(dists)

df_noe.columns = list(df_noe_exp_loop['Assignment'])
print(df_noe.shape)

df_noe.to_csv(f'calc_data/calc_dists_loop_bme.dat', sep='\t', header=False, float_format='%.2f')

(20100, 76)
